In [1]:
import pandas as pd
import re

#creating a pandas dataframe out of the csv file
traindf = pd.read_csv('../Data/08.Pubmed_PDFs_011620.csv')
testdf = pd.read_csv('../Data/04.Pubmed_PDFs_011320.csv')

out = [x[:13] for x in testdf["Identifiers"]]
#print(out)

subdf = traindf[~traindf["Identifiers"].isin(out)]
print(subdf.shape)
subdf

(88, 14)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,DateTime,Keyword,Title,URL,Abstract,Description,Details,ShortDetails,Resource,Type,Identifiers,Db,EntrezUID,Properties
0,011620-185708,"(""machine learning"" OR ""deep learning"" OR ""art...",Evaluating risk prediction models for adults w...,/pubmed/31940350,BACKGROUND: The ability to predict risk allows...,"Di Tanna GL, Wirtz H, Burrows KL, Globe G.",PloS one. doi: 10.1371/journal.pone.0224135. J...,PloS one. 2020,PubMed,Journal Article,PMID:31940350,pubmed,31940350,create date: 2020/01/15 | first author: Di Tan...
1,011620-185710,"(""machine learning"" OR ""deep learning"" OR ""art...",Automated Cardiovascular Pathology Assessment ...,/pubmed/31939003,Cardiac magnetic resonance imaging provides hi...,"Lindsey T, Lee JJ.",Journal of digital imaging. doi: 10.1007/s1027...,Journal of digital imaging. 2020,PubMed,Journal Article,PMID:31939003,pubmed,31939003,create date: 2020/01/14 | first author: Lindsey T
2,011620-185718,"(""machine learning"" OR ""deep learning"" OR ""art...",Physiological Assessment of Coronary Lesions i...,/pubmed/31938934,PURPOSE OF REVIEW: Physiological assessment of...,"Chowdhury M, Osborn EA.",Current treatment options in cardiovascular me...,Current treatment options in cardiovascular me...,PubMed,"Journal Article, Review",PMID:31938934,pubmed,31938934,create date: 2020/01/15 | first author: Chowdh...
3,011620-185722,"(""machine learning"" OR ""deep learning"" OR ""art...",Of Machines and Men: Intelligent Diagnosis and...,/pubmed/31938923,"Artificial Intelligence (AI), although well es...","Cockcroft J, Avolio A.",Current hypertension reports. doi: 10.1007/s11...,Current hypertension reports. 2020,PubMed,"Journal Article, Review",PMID:31938923,pubmed,31938923,create date: 2020/01/14 | first author: Cockcr...
4,011620-185724,"(""machine learning"" OR ""deep learning"" OR ""art...",Cytosine and adenine base editing of the brain...,/pubmed/31937940,The success of base editors for the study and ...,"Levy JM, Yeh WH, Pendse N, Davis JR, Hennessey...",Nature biomedical engineering. doi: 10.1038/s4...,Nature biomedical engineering. 2020,PubMed,Journal Article,PMID:31937940,pubmed,31937940,create date: 2020/01/14 | first author: Levy JM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,011620-194624,"(""machine learning"" OR ""deep learning"" OR ""art...",Lung Innate Lymphoid Cell Composition Is Alter...,/pubmed/31394048,Comment in\n Am J Respir Crit Care Med. 202...,"Monticelli LA, Diamond JM, Saenz SA, Tait Wojn...",American journal of respiratory and critical c...,American journal of respiratory and critical c...,PubMed,Journal Article,PMID:31394048,pubmed,31394048,create date: 2020/01/01 | first author: Montic...
974,011620-195203,"(""machine learning"" OR ""deep learning"" OR ""art...",Curcumin: a therapeutic strategy in cancers by...,/pubmed/31331376,Numerous studies have presented that curcumin ...,"Vallée A, Lecarpentier Y, Vallée JN.",Journal of experimental &amp; clinical cancer ...,Journal of experimental &amp; clinical cancer ...,PubMed,"Journal Article, Review",PMID:31331376,pubmed,31331376,create date: 2019/07/22 | first author: Vallée A
976,011620-195211,"(""machine learning"" OR ""deep learning"" OR ""art...",Extra Virgin Olive Oil Contains a Phenolic Inh...,/pubmed/31331073,The lysine-specific histone demethylase 1A (LS...,"Cuyàs E, Gumuzio J, Lozano-Sánchez J, Carreras...",Nutrients. pii: E1656. doi: 10.3390/nu11071656...,Nutrients. 2019,PubMed,Journal Article,PMID:31331073,pubmed,31331073,create date: 2019/07/19 | first author: Cuyàs E
977,011620-195219,"(""machine learning"" OR ""deep learning"" OR ""art...",Effect of Vasicinone against Paraquat-Induced ...,/pubmed/31331066,Vasicinone is a quinazoline alkaloid isolated ...,"Ju DT, Sivalingam K, Kuo WW, Ho TJ, Chang RL, ...",Nutrients. pii: E1655. doi: 10.3390/nu11071655...,Nutrients. 2019,PubMed,Journal Article,PMID:31331066,pubmed,31331066,create date: 2019/07/19 | first author: Ju DT


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
vectorizer = TfidfVectorizer(sublinear_tf=True, stop_words=text.ENGLISH_STOP_WORDS)
documentlist = subdf['Abstract'].values
X = vectorizer.fit_transform(documentlist)
words = vectorizer.get_feature_names()

In [53]:
doc =  4
wordlist = X[doc,:].nonzero()[1]
vallist = [X[doc, x] for x in wordlist]
topwords = sorted(range(len(vallist)), key=lambda i: vallist[i])[-200:]
[words[i] for i in wordlist[topwords]]

['study',
 'disease',
 'heart',
 'associated',
 'treatment',
 'use',
 'cell',
 'diseases',
 'therapeutic',
 'ability',
 'type',
 'targeted',
 'limited',
 'tissues',
 'efficient',
 'causes',
 'increasing',
 'genetic',
 'introduction',
 'mutations',
 'multiple',
 'application',
 '20',
 'report',
 'muscle',
 'facilitate',
 'liver',
 'neurodegenerative',
 'point',
 'deliver',
 'enable',
 'depends',
 'types',
 'viruses',
 'capacity',
 'length',
 'adenine',
 '59',
 'cortical',
 'skeletal',
 'mutation',
 'slowing',
 'neurodegeneration',
 'vectors',
 'tissue',
 'success',
 'adeno',
 'aav',
 'packaging',
 'precludes',
 'split',
 'cytosine',
 'reconstituted',
 'trans',
 'splicing',
 'inteins',
 'therapeutically',
 'efficiencies',
 'dosages',
 'unsorted',
 'retina',
 'corrects',
 'niemann',
 'pick',
 'ataxia',
 'lifespan',
 'relevant',
 'vivo',
 'brain',
 'mouse',
 '38',
 'dual',
 'optimized',
 'editing',
 'delivery',
 'editors',
 'aavs',
 'base']

In [54]:
[vallist[i] for i in topwords]

[0.04034532205181334,
 0.0459783742357598,
 0.0466999883519598,
 0.049839998484588886,
 0.05069779596505863,
 0.056661056409636486,
 0.05782689838935194,
 0.06173287179469072,
 0.06645229352398956,
 0.07241555396856741,
 0.07241555396856741,
 0.0748102863331974,
 0.07748736935362165,
 0.07748736935362165,
 0.08052239576380518,
 0.08052239576380518,
 0.08052239576380518,
 0.08402607662287498,
 0.08402607662287498,
 0.08402607662287498,
 0.08402607662287498,
 0.08817005152749834,
 0.08817005152749834,
 0.08817005152749834,
 0.08817005152749834,
 0.08817005152749834,
 0.09324186691255257,
 0.09324186691255257,
 0.09324186691255257,
 0.09978057418180593,
 0.09978057418180593,
 0.09978057418180593,
 0.09978057418180593,
 0.09978057418180593,
 0.09978057418180593,
 0.09978057418180593,
 0.09978057418180593,
 0.09978057418180593,
 0.09978057418180593,
 0.09978057418180593,
 0.09978057418180593,
 0.09978057418180593,
 0.09978057418180593,
 0.09978057418180593,
 0.10452283782704917,
 0.10899636

In [72]:
from rake_nltk import Rake #https://pypi.org/project/rake-nltk/
from nltk.corpus import stopwords 
r = Rake(stopwords = stopwords.words("english"), max_length = 5) # Uses stopwords for english from NLTK, and all puntuation characters.Please note that "hello" is not included in the list of stopwords.

a=r.extract_keywords_from_text(subdf['Abstract'][2])
b=r.get_ranked_phrases()
c=r.get_ranked_phrases_with_scores()
print(b)
print(c)

['term data demonstrating improved outcomes', 'determine lesion functional significance', 'beyond fractional flow reserve', 'ffrct ), invasive angiography', 'guided percutaneous coronary intervention', 'evaluating coronary stenosis physiology', 'coronary physiological assessment remains', 'invasive coronary ct', 'coronary physiological assessment', 'coronary artery disease', 'acute coronary syndromes', 'technical aspects involved', 'rest without hyperemia', 'residual ischemia post', 'optical coherence tomography', 'interventional cardiology toolbox', 'improve patient care', 'hyperemic pressure ratios', 'exciting technological future', 'enable detailed pre', 'artificial intelligence algorithms', 'alternative ffr approaches', 'assess coronary physiology', 'refine clinical practice', 'ffr ), non', 'physiological assessment', 'coronary physiology', 'remains underutilized', 'ffroct ),', 'ffrangio ),', 'current practice', 'clinical scenarios', 'well established', 'serial lesions', 'robust to

In [76]:
for 

(16.0, 'determine lesion functional significance')

In [65]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('../Data/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [74]:
model.similarity('cardiology', 'recent findings')
#model.most_similar(positive=['cardiology'])

KeyError: "word 'recent findings' not in vocabulary"

KeyError: "word 'svm' not in vocabulary"

In [111]:
from rake_nltk import Rake #https://pypi.org/project/rake-nltk/
from nltk.corpus import stopwords 
import time
start_time = time.time()
r = Rake(stopwords = stopwords.words("english"), max_length = 5) # Uses stopwords for english from NLTK, and all puntuation characters.Please note that "hello" is not included in the list of stopwords.
counter = 0
import csv
with open('../Data/Cardiolgydistance.csv', mode='w', encoding='UTF-8') as file:
    ewrite = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    ewrite.writerow(['Document', 'Phrase', 'RakeVal', 'SimVal'])
    for i in range(len(subdf['Abstract'])):
        a = r.extract_keywords_from_text(subdf['Abstract'][i])
        c=r.get_ranked_phrases_with_scores()
        for j in c:
            cosinevallist = []
            for k in j[1].split():
                try:
                    cval = model.similarity('cardiology', k)
                except:
                    cval = 0
                if cval != 0:
                    cosinevallist.append(cval)
            if len(cosinevallist) > 0:
                mcval = sum(cosinevallist) / len(cosinevallist)
            else:
                mcval = 0
            #print([counter, j[1], j[0], mcval])
            ewrite.writerow([counter, j[1], j[0], mcval])
    if (counter % 5 == 0):
        print("--- %s seconds ---" % (time.time() - start_time))
        print(counter)
    counter+=1
print("CSV done")
print("--- %s seconds ---" % (time.time() - start_time))


KeyError: 60

In [104]:
i = 1
from statistics import mean
a = r.extract_keywords_from_text(subdf['Abstract'][i])
c=r.get_ranked_phrases_with_scores()

    print([1, j[1], j[0], mcval])

[1, 'approach evaluated anonymized cardiac images', 23.666666666666668, 0.17590658068656922]
[1, 'advanced machine learning methods', 16.0, 0.05202275887131691]
[1, 'reproducible cardiac pathological assessment', 15.666666666666666, 0.253646831959486]
[1, 'fully automatic processing pipeline', 15.666666666666666, 0.07228710362687707]
[1, 'cardiac cine sequencing provides', 15.666666666666666, 0.24283761950209737]
[1, 'independent test set using', 15.5, 0.015956244431436062]
[1, 'initial processing element consists', 15.166666666666666, 0.03378185909241438]
[1, '20 patients per group', 14.5, 0.17405342496931553]
[1, 'ary pathology classification accuracy', 14.333333333333334, 0.14713991759344935]
[1, 'final processing element', 10.166666666666666, 0.023160612831513088]
[1, 'cardiac pathology classifier', 10.0, 0.4012519071499507]
[1, 'quantify cardiac measures', 9.666666666666666, 0.21810053630421558]
[1, 'manual cardiac evaluation', 9.666666666666666, 0.26817435150345165]
[1, 'training

ZeroDivisionError: division by zero

In [97]:
for i in c:
    
    print(i[1].split())

['approach', 'evaluated', 'anonymized', 'cardiac', 'images']
['advanced', 'machine', 'learning', 'methods']
['reproducible', 'cardiac', 'pathological', 'assessment']
['fully', 'automatic', 'processing', 'pipeline']
['cardiac', 'cine', 'sequencing', 'provides']
['independent', 'test', 'set', 'using']
['initial', 'processing', 'element', 'consists']
['20', 'patients', 'per', 'group']
['ary', 'pathology', 'classification', 'accuracy']
['final', 'processing', 'element']
['cardiac', 'pathology', 'classifier']
['quantify', 'cardiac', 'measures']
['manual', 'cardiac', 'evaluation']
['training', 'data', 'set']
['1', 'healthy', 'group']
['4', 'pathology', 'groups']
['rv', '),', 'respectively']
['performance', 'results', 'demonstrate']
['enabling', 'improved', 'extraction']
['cardiovascular', 'disease', 'staging']
['lv', '),', 'myocardium']
['combining', 'semantic', 'segmentation']
['disease', 'classification']
['100', 'patients']
['wise', 'segmentation']
['ventricular', 'cavities']
['university